In [4]:
!conda env export

name: exomepeak1
channels:
  - anaconda
  - python
  - r
  - bioconda
  - conda-forge
  - defaults
dependencies:
  - _libgcc_mutex=0.1=conda_forge
  - _openmp_mutex=4.5=1_llvm
  - _r-mutex=1.0.1=anacondar_1
  - backcall=0.2.0=py_0
  - binutils_impl_linux-64=2.34=h53a641e_5
  - binutils_linux-64=2.34=hc952b39_20
  - bioconductor-annotationdbi=1.42.1=r351_0
  - bioconductor-biobase=2.40.0=r351ha44fe06_1
  - bioconductor-biocgenerics=0.26.0=r351_0
  - bioconductor-biocparallel=1.14.2=r351h26a2512_0
  - bioconductor-biomart=2.36.1=r351_0
  - bioconductor-biostrings=2.48.0=r351h470a237_0
  - bioconductor-delayedarray=0.6.6=r351_0
  - bioconductor-exomepeak=2.14.0=r351_0
  - bioconductor-genomeinfodb=1.16.0=r351_0
  - bioconductor-genomeinfodbdata=1.2.1=r351_0
  - bioconductor-genomicalignments=1.16.0=r351h470a237_0
  - bioconductor-genomicfeatures=1.32.3=r351_0
  - bioconductor-genomicranges=1.32.7=r351h470a237_0
  - bioconductor-iranges=2.14.12=r351h470a237_0
  - bioconductor-rsamtools=1.3

In [41]:
Samples = [
    "S0008",
#     "S0009","S0014","S0017",
#     "S0025","S0042","S0057",
    "S0026","S0030",
    "S0085",
    "COV00075",
#     "COV00079", "COV00087",
#     "COV00084","COV00417",
    "COV00093","COV00106",
    "COV00397","COV00413",
    "COV00419","COV00422","COV00432"    
]

In [42]:
%reload_ext rpy2.ipython

In [43]:
%%R
library(GenomicFeatures)
# library(tidyverse)
library(exomePeak)

In [44]:
%%R -i Samples
GTF = '~/Projects/COVID19_m6A/virus/coronavirus_2_isolate_Wuhan-Hu-1.gff3'
IP = '_RIP'
INPUT = '_IN'

OUTPUT = 'exomepeak/virus'

Samples = unlist(Samples)

######################################## read meta ######################################$
IP_BAM = paste(Samples, IP, '.bam', sep='')
INPUT_BAM = paste(Samples, INPUT, '.bam', sep='')

In [45]:
%%R
######################################## functions ######################################
txdb <- makeTxDbFromGFF(GTF, organism=NA )

WINDOW = 50
STEP = 5
LENGTH = 200
ENRICH = 1
EXP = 'patient-as-replicates' # paste(Sample,'FDR', FDR,sep='_')

R[write to console]: Import genomic features from the file as a GRanges object ... 
R[write to console]: OK

R[write to console]: Prepare the 'metadata' data frame ... 
R[write to console]: OK

R[write to console]: Make the TxDb object ... 
R[write to console]: OK



In [46]:
%%R
txdb

TxDb object:
# Db type: TxDb
# Supporting package: GenomicFeatures
# Data source: ~/Projects/COVID19_m6A/virus/coronavirus_2_isolate_Wuhan-Hu-1.gff3
# Organism: NA
# Taxonomy ID: NA
# miRBase build ID: NA
# Genome: NA
# transcript_nrow: 9
# exon_nrow: 9
# cds_nrow: 9
# Db created by: GenomicFeatures package from Bioconductor
# Creation time: 2020-06-24 20:31:52 +0000 (Wed, 24 Jun 2020)
# GenomicFeatures version at creation time: 1.32.3
# RSQLite version at creation time: 2.1.1
# DBSCHEMAVERSION: 1.2


In [62]:
%%R
# setwd("/rumi/shams/abe/Projects/COVID19_m6A/virus_bam")
# list.files()
file.exists(INPUT_BAM)

 [1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE


In [63]:
%%R
setwd("/rumi/shams/abe/Projects/COVID19_m6A/virus_bam")

options(digits=5)
res <- exomepeak(
    TXDB = txdb,
    IP_BAM=IP_BAM,
    INPUT_BAM=INPUT_BAM,
#     OUTPUT_DIR=paste('..',OUTPUT,sep='/'),
#     EXPERIMENT_NAME=EXP,
    # options
    WINDOW_WIDTH = WINDOW,
    SLIDING_STEP = STEP,
    FRAGMENT_LENGTH = LENGTH,
    PEAK_CUTOFF_PVALUE = 1,
    PEAK_CUTOFF_FDR = 1, # as.double(FDR),
    FOLD_ENRICHMENT = ENRICH
)

saveRDS(res, paste('..', OUTPUT, EXP, 'results.rds', sep='/'))

R[write to console]: 'select()' returned 1:1 mapping between keys and columns



[1] "Divide transcriptome into chr-gene-batch sections ..."
[1] "Get Reads Count ..."
[1] "This step may take a few hours ..."
[1] "100 %"
[1] "Get all the peaks ..."
[1] "Get the consistent peaks ..."
[1] "---------------------------------"
[1] "The bam files used:"
[1] "12 IP replicate(s)"
[1] "12 Input replicate(s)"
[1] "---------------------------------"
[1] "Peak calling result: "
[1] "6 peaks detected on merged data."
[1] "Please check 'peak.bed/xls' under /rumi/shams/abe/Projects/COVID19_m6A/virus_bam/exomePeak_output"


R[write to console]: Error in `[.data.frame`(PEAK_RESULT[[2]], , 1) : 
  undefined columns selected

R[write to console]: In addition: 
R[write to console]: There were 34 warnings (use warnings() to see them)
R[write to console]: 




Error in `[.data.frame`(PEAK_RESULT[[2]], , 1) : 
  undefined columns selected


In [51]:
%%R 
setwd("../")

In [52]:
%%R
warnings()

Warning messages:
1: In makeTxDbFromGRanges(gr, metadata = metadata) :
  The following transcripts were dropped because their exon ranks could
  not be inferred (either because the exons are not on the same
  chromosome/strand or because they are not separated by introns):
  gene-orf1ab
2: In is.na(PARAMETERS$TXDB) :
  is.na() applied to non-(list or vector) of type 'S4'
3: In is.na(PARAMETERS$TXDB) :
  is.na() applied to non-(list or vector) of type 'S4'
4: In is.na(PARAMETERS$TXDB) :
  is.na() applied to non-(list or vector) of type 'S4'
5: In is.na(PARAMETERS$TXDB) :
  is.na() applied to non-(list or vector) of type 'S4'
6: In pbinom(IP - 1, total, p, lower.tail = FALSE, log.p = TRUE) :
  pbeta(*, log.p=TRUE) -> bpser(a=1533, b=33, x=0.559034,...) underflow to -Inf
7: In pbinom(IP - 1, total, p, lower.tail = FALSE, log.p = TRUE) :
  pbeta(*, log.p=TRUE) -> bpser(a=1504, b=32, x=0.559034,...) underflow to -Inf
8: In pbinom(IP - 1, total, p, lower.tail = FALSE, log.p = TRUE) :
  pbeta

In [55]:
!ls exomepeak/virus/patient-as-replicates/

con_peak.bed  con_peak.xls  exomePeak.Rdata  peak.bed  peak.xls


In [29]:
%%R 
res %>% data.frame %>% head 

R[write to console]: Error in eval(lhs, parent, parent) : object 'res' not found
Calls: <Anonymous> ... <Anonymous> -> <Anonymous> -> withVisible -> %>% -> eval -> eval




Error in eval(lhs, parent, parent) : object 'res' not found
Calls: <Anonymous> ... <Anonymous> -> <Anonymous> -> withVisible -> %>% -> eval -> eval
